# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

import pickle

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/rzsyed/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rzsyed/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /Users/rzsyed/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
# load data from database
engine = create_engine('sqlite:///clean_db.db')
df = pd.read_sql_table('clean_table', con = engine)
X = df['message']
y = df.iloc[:,4:]

In [6]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    tokens = word_tokenize(text.lower())
    
    stop_words = set(stopwords.words('english'))
    filtered_sentence = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in filtered_sentence:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer = tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = .2,random_state = 1)
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fbba1038430>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
y_pred = pipeline.predict(X_test)

In [ ]:
import numpy as np
labels = np.unique(y_pred)
target_names = y.columns

In [20]:
def test_model(y_test, y_pred):
    """
    Function to iterate through columns and call sklearn classification report on each.
    """
    for index, column in enumerate(y_test):
        print(column, classification_report(y_test[column], y_pred[:, index]))

In [21]:
test_model(y_test, y_pred)

related               precision    recall  f1-score   support

           0       0.75      0.37      0.49      1241
           1       0.82      0.96      0.89      3963
           2       0.67      0.15      0.24        40

    accuracy                           0.81      5244
   macro avg       0.75      0.49      0.54      5244
weighted avg       0.80      0.81      0.79      5244

request               precision    recall  f1-score   support

           0       0.90      0.98      0.94      4325
           1       0.84      0.50      0.62       919

    accuracy                           0.90      5244
   macro avg       0.87      0.74      0.78      5244
weighted avg       0.89      0.90      0.88      5244

offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5221
           1       0.00      0.00      0.00        23

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      524

/Users/rzsyed/.conda/envs/recsys/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rzsyed/.conda/envs/recsys/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rzsyed/.conda/envs/recsys/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/

storm               precision    recall  f1-score   support

           0       0.94      0.99      0.97      4741
           1       0.82      0.43      0.56       503

    accuracy                           0.94      5244
   macro avg       0.88      0.71      0.76      5244
weighted avg       0.93      0.94      0.93      5244

fire               precision    recall  f1-score   support

           0       0.99      1.00      0.99      5181
           1       0.00      0.00      0.00        63

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.98      0.99      0.98      5244

earthquake               precision    recall  f1-score   support

           0       0.97      0.99      0.98      4759
           1       0.89      0.74      0.81       485

    accuracy                           0.97      5244
   macro avg       0.93      0.87      0.89      5244
weighted avg       0.97      0.97      0.97      524

/Users/rzsyed/.conda/envs/recsys/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
(y_pred == y_test).mean()

related                   0.814073
request                   0.895118
offer                     0.995614
aid_related               0.770404
medical_help              0.917239
medical_products          0.949847
search_and_rescue         0.974828
security                  0.983791
military                  0.971777
child_alone               1.000000
water                     0.953089
food                      0.939931
shelter                   0.931541
clothing                  0.986079
money                     0.979024
missing_people            0.987986
refugees                  0.968345
death                     0.962815
other_aid                 0.867849
infrastructure_related    0.933448
transport                 0.958238
buildings                 0.948894
electricity               0.980549
tools                     0.993516
hospitals                 0.989321
shops                     0.995614
aid_centers               0.989512
other_infrastructure      0.953089
weather_related     

In [22]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fbba1038430>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fbba1038430>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

### 6. Improve your model
Use grid search to find better parameters. 

In [41]:
 parameters = {
        'clf__estimator__n_estimators': [10,20],
        # 'clf__estimator__min_samples_split': [2, 3, 4],
        'clf__n_jobs': [-1]
    }

cv = GridSearchCV(pipeline,param_grid=parameters)
cv.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fbba1038430>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20],
                         'clf__n_jobs': [-1]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [42]:
grid_y_pred = cv.predict(X_test)
print((grid_y_pred == y_test).mean())
test_model(y_test, grid_y_pred)

related                   0.806827
request                   0.883677
offer                     0.995614
aid_related               0.756293
medical_help              0.915332
medical_products          0.951564
search_and_rescue         0.975019
security                  0.983600
military                  0.971205
child_alone               1.000000
water                     0.949657
food                      0.938024
shelter                   0.931350
clothing                  0.986270
money                     0.979214
missing_people            0.987986
refugees                  0.968726
death                     0.961480
other_aid                 0.866895
infrastructure_related    0.933448
transport                 0.956903
buildings                 0.950992
electricity               0.980740
tools                     0.993516
hospitals                 0.989321
shops                     0.995614
aid_centers               0.989512
other_infrastructure      0.953471
weather_related     

/Users/rzsyed/.conda/envs/recsys/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rzsyed/.conda/envs/recsys/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rzsyed/.conda/envs/recsys/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/

weather_related               precision    recall  f1-score   support

           0       0.87      0.95      0.91      3793
           1       0.84      0.63      0.72      1451

    accuracy                           0.86      5244
   macro avg       0.85      0.79      0.82      5244
weighted avg       0.86      0.86      0.86      5244

floods               precision    recall  f1-score   support

           0       0.95      1.00      0.97      4819
           1       0.89      0.40      0.55       425

    accuracy                           0.95      5244
   macro avg       0.92      0.70      0.76      5244
weighted avg       0.94      0.95      0.94      5244

storm               precision    recall  f1-score   support

           0       0.94      0.99      0.96      4741
           1       0.78      0.44      0.56       503

    accuracy                           0.93      5244
   macro avg       0.86      0.71      0.76      5244
weighted avg       0.93      0.93      0.93  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
filename = 'model.pkl'
pickle.dump(cv,open(filename,'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.